In [1]:
import os
os.chdir("/home/atulgang/Thesis")


In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
import nltk
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models
import gensim
import glob2
from itertools import chain
import os
import numpy as np
import itertools
import cPickle as pickle
from  scipy.stats import rankdata
import copy
from openpyxl import Workbook
from openpyxl.compat import range
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
import pandas as pd
from scipy.stats import hmean
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

2018-05-03 19:02:11,887 : INFO : 'pattern' package not found; tag filters are not available for English


In [4]:

# WordNet only cares about 5 parts of speech.
# The other parts of speech will be tagged as nouns.

part = {
    'N' : 'n',
    'V' : 'v',
    'J' : 'a',
    'S' : 's',
    'R' : 'r'
}

wnl = WordNetLemmatizer()

def convert_tag(penn_tag):
    '''
    convert_tag() accepts the **first letter** of a Penn part-of-speech tag,
    then uses a dict lookup to convert it to the appropriate WordNet tag.
    '''
    if penn_tag in part.keys():
        return part[penn_tag]
    else:
        # other parts of speech will be tagged as nouns
        return 'n'


def tag_and_lem(element):
    '''
    tag_and_lem() accepts a token list, tags, converts tags,
    lemmatizes, and returns a string
    '''
    # list of tuples [('token', 'tag'), ('token2', 'tag2')...]
    sent = pos_tag(element) # must tag in context
    return [ wnl.lemmatize(sent[k][0], convert_tag(sent[k][1][0])) for k in range(len(sent))]

In [5]:
# ignores everything except english alphabet and  
def only_alphabet(text):
    return ''.join(i for i in text if (ord(i)<123 and ord(i)>96) or (ord(i)<91 and ord(i)>64) or ord(i)==32) 

full_filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        full_filenames.append(filename)
            
filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        filenames.append(os.path.basename(filename))

In [6]:
en_stop = set(get_stop_words('en'))

# "yield" for each file return token list  i.e list of lists
def files_to_tokens(glob_filenames):
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        f = open(filename)
        #print filename
        # read the whole file as lowercase string
        string = (f.read()).lower()
        try:
            temp = word_tokenize(string)
        except:
            string = string.decode("latin-1")
            temp = word_tokenize(string)
        
        tokens = []
        # tokenize that string
        for word in temp:
            w =  only_alphabet(word).lower()
            if w not in en_stop:
                if w:
                    tokens.append(w)
        tokens = tag_and_lem(tokens)
        yield tokens
        f.close()

        
# yields token list for files specific to courts; needed for creating dictionaries
class texts:
    def DCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt')
    def NCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt')
    def SCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt')
    def DelhiHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt')
    def JharkhandHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt')
    def JodhpurHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt')
    def KolkataHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt')
    def SupremeCourt(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt')
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())

# yields bow for each file - tuples id,fq ; needed to train models   
class my_corpus:    
    def DCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/*.txt'):
            yield dictionary.doc2bow(text)
    def NCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)            
    def SCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def DelhiHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JharkhandHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JodhpurHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def KolkataHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def SupremeCourt(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt'):
            yield dictionary.doc2bow(text)
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())
        
        

In [7]:
#################################### laod the dictinary
dictionary = corpora.Dictionary.load('dict/lem10')

2018-05-03 19:02:33,328 : INFO : loading Dictionary object from dict/lem10
2018-05-03 19:02:33,552 : INFO : loaded dict/lem10


In [8]:
######################################## load BOW
corpus = corpora.MmCorpus('corpus/bow_lem10.mm')

2018-05-03 19:02:33,702 : INFO : loaded corpus index from corpus/bow_lem10.mm.index
2018-05-03 19:02:33,703 : INFO : initializing cython corpus reader from corpus/bow_lem10.mm
2018-05-03 19:02:33,754 : INFO : accepted corpus with 351985 documents, 169716 features, 99651009 non-zero entries


In [9]:
%%time
from gensim.models import ldamodel
lda_model = ldamodel.LdaModel(corpus, num_topics=200, alpha = 'auto', id2word = dictionary, passes = 3,  minimum_probability =0.0)

2018-05-03 19:03:25,895 : INFO : using autotuned alpha, starting with [0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005

/home/atulgang/anaconda3/envs/python2/lib/python2.7/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)
2018-05-03 19:03:51,684 : INFO : merging changes from 2000 documents into a model of 351985 documents
2018-05-03 19:03:55,344 : INFO : topic #3 (0.005): 0.028*"crop" + 0.022*"yield" + 0.018*"inview" + 0.015*"fact" + 0.015*"complainant" + 0.014*"op" + 0.013*"land" + 0.013*"area" + 0.011*"insurance" + 0.011*"loss"
2018-05-03 19:03:55,346 : INFO : topic #73 (0.005): 0.034*"complainant" + 0.033*"tile" + 0.033*"opposite" + 0.021*"opno" + 0.021*"licence" + 0.019*"basavaraj" + 0.016*"basappa" + 0.012*"dt" + 0.010*"eye" + 0.009*"say"
2018-05-03 19:03:55,347 : INFO : topic #173 (0.008): 0.053*"complainant" + 0.021*"amount" + 0.019*"opponent" + 0.018*"r" + 0.018*"complaint" + 0.017*"pay" + 0.012*"party" + 0.011*"interest" + 0.010*"opposite" + 0.009*"s"
2018-05-03 19:03:55,349 : INFO : topic #136 (0.008): 0.063*"compla

2018-05-03 19:04:20,670 : INFO : merging changes from 2000 documents into a model of 351985 documents
2018-05-03 19:04:24,291 : INFO : topic #175 (0.005): 0.057*"petitioner" + 0.043*"kar" + 0.023*"crop" + 0.023*"ilr" + 0.022*"law" + 0.020*"complainant" + 0.015*"person" + 0.015*"honble" + 0.014*"commission" + 0.014*"insurance"
2018-05-03 19:04:24,292 : INFO : topic #36 (0.005): 0.042*"article" + 0.035*"value" + 0.033*"complainant" + 0.017*"transport" + 0.017*"loss" + 0.016*"carrier" + 0.015*"seize" + 0.015*"miss" + 0.012*"fact" + 0.009*"transit"
2018-05-03 19:04:24,294 : INFO : topic #123 (0.009): 0.066*"complainant" + 0.040*"kvg" + 0.035*"krishi" + 0.029*"maize" + 0.028*"produce" + 0.020*"crop" + 0.019*"complaint" + 0.019*"hence" + 0.016*"tq" + 0.012*"c"
2018-05-03 19:04:24,295 : INFO : topic #170 (0.009): 0.112*"adv" + 0.110*"sri" + 0.076*"ro" + 0.047*"circle" + 0.041*"road" + 0.040*"bangalore" + 0.037*"bhavan" + 0.035*"major" + 0.035*"cell" + 0.024*"manager"
2018-05-03 19:04:24,297 :

2018-05-03 19:04:51,719 : INFO : topic #3 (0.005): 0.206*"syno" + 0.045*"claim" + 0.040*"crop" + 0.030*"grow" + 0.027*"produce" + 0.023*"complaint" + 0.021*"complainant" + 0.020*"show" + 0.019*"yield" + 0.019*"government"
2018-05-03 19:04:51,720 : INFO : topic #17 (0.005): 0.074*"corroborates" + 0.072*"stranger" + 0.057*"freezer" + 0.033*"clamp" + 0.027*"complainant" + 0.019*"nd" + 0.017*"flow" + 0.016*"st" + 0.011*"party" + 0.010*"op"
2018-05-03 19:04:51,722 : INFO : topic #148 (0.011): 0.053*"complainant" + 0.022*"r" + 0.015*"service" + 0.015*"complaint" + 0.012*"llb" + 0.010*"say" + 0.010*"notice" + 0.010*"thecomplainant" + 0.010*"point" + 0.009*"respondent"
2018-05-03 19:04:51,723 : INFO : topic #189 (0.011): 0.033*"complaint" + 0.027*"forum" + 0.025*"complainant" + 0.020*"file" + 0.016*"jurisdiction" + 0.012*"ccno" + 0.010*"point" + 0.010*"thecomplainant" + 0.010*"branch" + 0.009*"district"
2018-05-03 19:04:51,725 : INFO : topic #40 (0.015): 0.259*"op" + 0.045*"complainant" + 0.01

2018-05-03 19:05:22,921 : INFO : topic #17 (0.005): 0.151*"flow" + 0.124*"stranger" + 0.094*"freezer" + 0.035*"corroborates" + 0.023*"complainant" + 0.013*"clamp" + 0.013*"iv" + 0.012*"party" + 0.009*"st" + 0.009*"consider"
2018-05-03 19:05:22,923 : INFO : topic #130 (0.014): 0.174*"party" + 0.144*"opposite" + 0.051*"complainant" + 0.011*"complaint" + 0.010*"pay" + 0.010*"oppositeparty" + 0.008*"service" + 0.008*"per" + 0.007*"file" + 0.007*"r"
2018-05-03 19:05:22,924 : INFO : topic #148 (0.015): 0.048*"complainant" + 0.022*"ex" + 0.020*"r" + 0.018*"thecomplainant" + 0.014*"complaint" + 0.013*"service" + 0.013*"llb" + 0.010*"notice" + 0.010*"member" + 0.009*"point"
2018-05-03 19:05:22,926 : INFO : topic #40 (0.017): 0.267*"op" + 0.046*"complainant" + 0.020*"complaint" + 0.018*"r" + 0.011*"pay" + 0.009*"file" + 0.008*"member" + 0.008*"service" + 0.006*"hence" + 0.006*"deficiency"
2018-05-03 19:05:22,991 : INFO : topic diff=inf, rho=0.353553
2018-05-03 19:05:23,299 : INFO : PROGRESS: pas

...

2018-05-03 20:31:09,530 : INFO : topic #148 (0.054): nan*"judgementorder" + nan*"actionlet" + nan*"balai" + nan*"froud" + nan*"krishnapada" + nan*"khokan" + nan*"bhasskar" + nan*"nirod" + nan*"bishnupada" + nan*"uniot"
2018-05-03 20:31:09,531 : INFO : topic #130 (0.072): nan*"judgementorder" + nan*"actionlet" + nan*"balai" + nan*"froud" + nan*"krishnapada" + nan*"khokan" + nan*"bhasskar" + nan*"nirod" + nan*"bishnupada" + nan*"uniot"
2018-05-03 20:31:09,592 : INFO : topic diff=nan, rho=0.074953
2018-05-03 20:31:24,692 : INFO : nan per-word bound, nan perplexity estimate based on a held-out corpus of 2000 documents with 2938476 words
2018-05-03 20:31:24,693 : INFO : PROGRESS: pass 2, at document #340000/351985
2018-05-03 20:31:34,182 : WARNING : updated prior not positive
2018-05-03 20:31:34,182 : INFO : optimized alpha [0.00760528, 0.0069462126, 0.0076380065, 0.00454008, 0.005741742, 0.009970834, 0.015699908, 0.0052111875, 0.005549969, 0.0052760374, 0.027318958, 0.005577444, 0.00607062

2018-05-03 20:31:47,483 : INFO : topic #148 (0.054): nan*"judgementorder" + nan*"actionlet" + nan*"balai" + nan*"froud" + nan*"krishnapada" + nan*"khokan" + nan*"bhasskar" + nan*"nirod" + nan*"bishnupada" + nan*"uniot"
2018-05-03 20:31:47,485 : INFO : topic #130 (0.072): nan*"judgementorder" + nan*"actionlet" + nan*"balai" + nan*"froud" + nan*"krishnapada" + nan*"khokan" + nan*"bhasskar" + nan*"nirod" + nan*"bishnupada" + nan*"uniot"
2018-05-03 20:31:47,543 : INFO : topic diff=nan, rho=0.074953
2018-05-03 20:31:47,993 : INFO : PROGRESS: pass 2, at document #344000/351985
2018-05-03 20:31:58,535 : WARNING : updated prior not positive
2018-05-03 20:31:58,535 : INFO : optimized alpha [0.00760528, 0.0069462126, 0.0076380065, 0.00454008, 0.005741742, 0.009970834, 0.015699908, 0.0052111875, 0.005549969, 0.0052760374, 0.027318958, 0.005577444, 0.0060706283, 0.011890469, 0.0074725715, 0.0072670276, 0.0068196207, 0.0048286333, 0.0076915016, 0.01021521, 0.011768547, 0.007159948, 0.006610962, 0.0

2018-05-03 20:32:10,486 : INFO : topic #130 (0.072): nan*"judgementorder" + nan*"actionlet" + nan*"balai" + nan*"froud" + nan*"krishnapada" + nan*"khokan" + nan*"bhasskar" + nan*"nirod" + nan*"bishnupada" + nan*"uniot"
2018-05-03 20:32:10,543 : INFO : topic diff=nan, rho=0.074953
2018-05-03 20:32:10,917 : INFO : PROGRESS: pass 2, at document #348000/351985
2018-05-03 20:32:19,793 : WARNING : updated prior not positive
2018-05-03 20:32:19,794 : INFO : optimized alpha [0.00760528, 0.0069462126, 0.0076380065, 0.00454008, 0.005741742, 0.009970834, 0.015699908, 0.0052111875, 0.005549969, 0.0052760374, 0.027318958, 0.005577444, 0.0060706283, 0.011890469, 0.0074725715, 0.0072670276, 0.0068196207, 0.0048286333, 0.0076915016, 0.01021521, 0.011768547, 0.007159948, 0.006610962, 0.0070741996, 0.00902805, 0.0058076964, 0.009922782, 0.0053010765, 0.005396866, 0.010541478, 0.013206296, 0.007998491, 0.0064773313, 0.009687916, 0.007254157, 0.0060064066, 0.007052737, 0.0067389426, 0.008439226, 0.0073143

2018-05-03 20:32:32,116 : INFO : topic diff=nan, rho=0.074953
2018-05-03 20:32:48,475 : INFO : nan per-word bound, nan perplexity estimate based on a held-out corpus of 1985 documents with 3594211 words
2018-05-03 20:32:48,476 : INFO : PROGRESS: pass 2, at document #351985/351985
2018-05-03 20:32:58,594 : WARNING : updated prior not positive
2018-05-03 20:32:58,595 : INFO : optimized alpha [0.00760528, 0.0069462126, 0.0076380065, 0.00454008, 0.005741742, 0.009970834, 0.015699908, 0.0052111875, 0.005549969, 0.0052760374, 0.027318958, 0.005577444, 0.0060706283, 0.011890469, 0.0074725715, 0.0072670276, 0.0068196207, 0.0048286333, 0.0076915016, 0.01021521, 0.011768547, 0.007159948, 0.006610962, 0.0070741996, 0.00902805, 0.0058076964, 0.009922782, 0.0053010765, 0.005396866, 0.010541478, 0.013206296, 0.007998491, 0.0064773313, 0.009687916, 0.007254157, 0.0060064066, 0.007052737, 0.0067389426, 0.008439226, 0.007314397, 0.018725805, 0.0131586185, 0.008015156, 0.005628546, 0.009402399, 0.008074

CPU times: user 3h 43min 47s, sys: 5h 59min 6s, total: 9h 42min 53s
Wall time: 1h 29min 34s


In [10]:
lda_model.save('py_models/lda_200.model')

2018-05-03 20:33:00,013 : INFO : saving LdaState object under py_models/lda_200.model.state, separately None
2018-05-03 20:33:00,014 : INFO : storing np array 'sstats' to py_models/lda_200.model.state.sstats.npy
2018-05-03 20:33:00,078 : INFO : saved py_models/lda_200.model.state
2018-05-03 20:33:00,305 : INFO : saving LdaModel object under py_models/lda_200.model, separately ['expElogbeta', 'sstats']
2018-05-03 20:33:00,306 : INFO : not storing attribute id2word
2018-05-03 20:33:00,306 : INFO : storing np array 'expElogbeta' to py_models/lda_200.model.expElogbeta.npy
2018-05-03 20:33:00,359 : INFO : not storing attribute state
2018-05-03 20:33:00,360 : INFO : not storing attribute dispatcher
2018-05-03 20:33:00,361 : INFO : saved py_models/lda_200.model


In [11]:
lda_model = gensim.models.LdaModel.load('py_models/lda_200.model')

2018-05-03 20:33:00,365 : INFO : loading LdaModel object from py_models/lda_200.model
2018-05-03 20:33:00,366 : INFO : loading expElogbeta from py_models/lda_200.model.expElogbeta.npy with mmap=None
2018-05-03 20:33:00,407 : INFO : setting ignored attribute id2word to None
2018-05-03 20:33:00,408 : INFO : setting ignored attribute state to None
2018-05-03 20:33:00,409 : INFO : setting ignored attribute dispatcher to None
2018-05-03 20:33:00,409 : INFO : loaded py_models/lda_200.model
2018-05-03 20:33:00,410 : INFO : loading LdaState object from py_models/lda_200.model.state
2018-05-03 20:33:00,410 : INFO : loading sstats from py_models/lda_200.model.state.sstats.npy with mmap=None
2018-05-03 20:33:00,453 : INFO : loaded py_models/lda_200.model.state


In [12]:
lda_model.show_topics(-1)

[(0,
  u'nan*"judgementorder" + nan*"actionlet" + nan*"balai" + nan*"froud" + nan*"krishnapada" + nan*"khokan" + nan*"bhasskar" + nan*"nirod" + nan*"bishnupada" + nan*"uniot"'),
 (1,
  u'nan*"judgementorder" + nan*"actionlet" + nan*"balai" + nan*"froud" + nan*"krishnapada" + nan*"khokan" + nan*"bhasskar" + nan*"nirod" + nan*"bishnupada" + nan*"uniot"'),
 (2,
  u'nan*"judgementorder" + nan*"actionlet" + nan*"balai" + nan*"froud" + nan*"krishnapada" + nan*"khokan" + nan*"bhasskar" + nan*"nirod" + nan*"bishnupada" + nan*"uniot"'),
 (3,
  u'nan*"judgementorder" + nan*"actionlet" + nan*"balai" + nan*"froud" + nan*"krishnapada" + nan*"khokan" + nan*"bhasskar" + nan*"nirod" + nan*"bishnupada" + nan*"uniot"'),
 (4,
  u'nan*"judgementorder" + nan*"actionlet" + nan*"balai" + nan*"froud" + nan*"krishnapada" + nan*"khokan" + nan*"bhasskar" + nan*"nirod" + nan*"bishnupada" + nan*"uniot"'),
 (5,
  u'nan*"judgementorder" + nan*"actionlet" + nan*"balai" + nan*"froud" + nan*"krishnapada" + nan*"khokan"

In [ ]:
%%time
theta, _ = lda_model.inference(corpus)
theta /= theta.sum(axis=1)[:, None]

In [ ]:
#Save
with open('py_models/thetalda_200.pkl', 'wb') as output:
    pickle.dump(theta, output, pickle.HIGHEST_PROTOCOL)

In [ ]:
# load 
with open('py_models/thetalda_200.pkl', 'rb') as ip:
    theta = pickle.load(ip)



In [ ]:
%%time
topics_terms = lda_model.state.get_lambda()
topics_terms = np.apply_along_axis(lambda x: x/x.sum(),1,topics_terms)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gensim.matutils as gm

In [ ]:
# get the query truth in dictionary
query_truth = {}
for i in range(1,11):
    query_truth[str(i)]=[]
    
    
f = open('LegalAdhocTask/Consumer.qrels')
lines = [line.rstrip('\n').split("\t") for line in f]
for line in lines:
    del line[1]
    query_truth[line[0]].append(line[1:])


In [ ]:
def sim_matrix(query, sim_fn):
    docs = query_truth[str(query)]
    docs.sort(key = lambda x : -int(x[1]))
    n = 0
    for i in docs:
        n = n + int(i[1])
    s  = len(docs)
    m = np.zeros((s,s))
    for i in xrange(s):
        for j in xrange(i + 1, s):
            v1 = theta[filenames.index(docs[i][0] + '.txt')]
            v2 = theta[filenames.index(docs[j][0] + '.txt')]
            if sim_fn == gm.cossim:
                v1 = gm.dense2vec(v1)
                v2 = gm.dense2vec(v2)
            m[i, j] = (1 - sim_fn(v1, v2))
    #print m
    print 'Query', query
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 10) 
    ax.matshow(m, cmap=plt.cm.Blues)
    ax.hlines(n -0.5, n -0.5, s - 0.5, lw=2, color='r')
    ax.vlines(n -0.5, -0.5, n - 0.5, lw=2, color='r')
    plt.show()


In [ ]:
%%time
for query in xrange(1, 11):
    sim_matrix(query, gm.hellinger)

In [11]:
%%time
# tfidf wrapper coprus
# doesn't take time - yield
tfidf_corpus = tfidf[corpus]

# actual tfidf corpus serialized
corpora.MmCorpus.serialize('corpus/tfidf_lem10.mm', tfidf_corpus)


2018-04-29 17:43:05,635 : INFO : storing corpus in Matrix Market format to corpus/tfidf_lem10.mm
2018-04-29 17:43:05,636 : INFO : saving sparse matrix to corpus/tfidf_lem10.mm
2018-04-29 17:43:05,638 : INFO : PROGRESS: saving document #0
2018-04-29 17:43:07,478 : INFO : PROGRESS: saving document #1000
2018-04-29 17:43:08,806 : INFO : PROGRESS: saving document #2000
2018-04-29 17:43:10,353 : INFO : PROGRESS: saving document #3000
2018-04-29 17:43:12,209 : INFO : PROGRESS: saving document #4000
2018-04-29 17:43:14,512 : INFO : PROGRESS: saving document #5000
2018-04-29 17:43:16,109 : INFO : PROGRESS: saving document #6000
2018-04-29 17:43:17,784 : INFO : PROGRESS: saving document #7000
2018-04-29 17:43:20,392 : INFO : PROGRESS: saving document #8000
2018-04-29 17:43:22,222 : INFO : PROGRESS: saving document #9000
2018-04-29 17:43:24,589 : INFO : PROGRESS: saving document #10000
2018-04-29 17:43:26,605 : INFO : PROGRESS: saving document #11000
2018-04-29 17:43:28,452 : INFO : PROGRESS: sa

2018-04-29 17:46:51,044 : INFO : PROGRESS: saving document #122000
2018-04-29 17:46:53,084 : INFO : PROGRESS: saving document #123000
2018-04-29 17:46:55,276 : INFO : PROGRESS: saving document #124000
2018-04-29 17:46:57,913 : INFO : PROGRESS: saving document #125000
2018-04-29 17:47:00,236 : INFO : PROGRESS: saving document #126000
2018-04-29 17:47:02,136 : INFO : PROGRESS: saving document #127000
2018-04-29 17:47:03,663 : INFO : PROGRESS: saving document #128000
2018-04-29 17:47:05,237 : INFO : PROGRESS: saving document #129000
2018-04-29 17:47:06,445 : INFO : PROGRESS: saving document #130000
2018-04-29 17:47:07,560 : INFO : PROGRESS: saving document #131000
2018-04-29 17:47:08,696 : INFO : PROGRESS: saving document #132000
2018-04-29 17:47:09,816 : INFO : PROGRESS: saving document #133000
2018-04-29 17:47:11,106 : INFO : PROGRESS: saving document #134000
2018-04-29 17:47:12,167 : INFO : PROGRESS: saving document #135000
2018-04-29 17:47:13,224 : INFO : PROGRESS: saving document #13

2018-04-29 17:50:06,732 : INFO : PROGRESS: saving document #245000
2018-04-29 17:50:07,299 : INFO : PROGRESS: saving document #246000
2018-04-29 17:50:07,982 : INFO : PROGRESS: saving document #247000
2018-04-29 17:50:08,613 : INFO : PROGRESS: saving document #248000
2018-04-29 17:50:09,242 : INFO : PROGRESS: saving document #249000
2018-04-29 17:50:09,958 : INFO : PROGRESS: saving document #250000
2018-04-29 17:50:10,650 : INFO : PROGRESS: saving document #251000
2018-04-29 17:50:11,287 : INFO : PROGRESS: saving document #252000
2018-04-29 17:50:11,870 : INFO : PROGRESS: saving document #253000
2018-04-29 17:50:12,399 : INFO : PROGRESS: saving document #254000
2018-04-29 17:50:12,912 : INFO : PROGRESS: saving document #255000
2018-04-29 17:50:13,375 : INFO : PROGRESS: saving document #256000
2018-04-29 17:50:13,831 : INFO : PROGRESS: saving document #257000
2018-04-29 17:50:14,277 : INFO : PROGRESS: saving document #258000
2018-04-29 17:50:14,731 : INFO : PROGRESS: saving document #25

CPU times: user 9min 21s, sys: 3.45 s, total: 9min 24s
Wall time: 9min 26s


In [12]:
##################################### load tfidf corpus
tfidf_corpus = corpora.MmCorpus('corpus/tfidf_lem10.mm')

2018-04-29 17:52:32,381 : INFO : loaded corpus index from corpus/tfidf_lem10.mm.index
2018-04-29 17:52:32,382 : INFO : initializing cython corpus reader from corpus/tfidf_lem10.mm
2018-04-29 17:52:32,382 : INFO : accepted corpus with 351985 documents, 169716 features, 99651009 non-zero entries


In [13]:
# get the query truth in dictionary
query_truth = {}
for i in range(1,11):
    query_truth[str(i)]=[]
    
    
f = open('LegalAdhocTask/Consumer.qrels')
lines = [line.rstrip('\n').split("\t") for line in f]
for line in lines:
    del line[1]
    query_truth[line[0]].append(line[1:])


In [14]:
%%time
# create index - Tfidf
tfidf_index = gensim.similarities.Similarity('py_models/tfidf_lem10.index',tfidf_corpus,len(dictionary))

# save index - tfidf
gensim.similarities.Similarity.save(tfidf_index,'py_models/TFIDF_lem10_index')


2018-04-29 17:52:32,562 : INFO : starting similarity index under py_models/tfidf_lem10.index
2018-04-29 17:52:37,110 : INFO : PROGRESS: fresh_shard size=10000
2018-04-29 17:52:42,448 : INFO : PROGRESS: fresh_shard size=20000
2018-04-29 17:52:47,068 : INFO : PROGRESS: fresh_shard size=30000
2018-04-29 17:52:48,356 : INFO : creating sparse index
2018-04-29 17:52:48,356 : INFO : creating sparse matrix from corpus
2018-04-29 17:52:48,357 : INFO : PROGRESS: at document #0/32768
2018-04-29 17:52:53,527 : INFO : PROGRESS: at document #10000/32768
2018-04-29 17:52:59,000 : INFO : PROGRESS: at document #20000/32768
2018-04-29 17:53:04,175 : INFO : PROGRESS: at document #30000/32768
2018-04-29 17:53:05,426 : INFO : created <32768x169716 sparse matrix of type '<type 'numpy.float32'>'
	with 10888040 stored elements in Compressed Sparse Row format>
2018-04-29 17:53:05,427 : INFO : creating sparse shard #0
2018-04-29 17:53:05,428 : INFO : saving index shard to py_models/tfidf_lem10.index.0
2018-04-2

2018-04-29 17:55:42,667 : INFO : saved py_models/tfidf_lem10.index.5
2018-04-29 17:55:42,667 : INFO : loading SparseMatrixSimilarity object from py_models/tfidf_lem10.index.5
2018-04-29 17:55:42,668 : INFO : loading index from py_models/tfidf_lem10.index.5.index.npy with mmap=r
2018-04-29 17:55:42,669 : INFO : loaded py_models/tfidf_lem10.index.5
2018-04-29 17:55:42,703 : INFO : PROGRESS: fresh_shard size=0
2018-04-29 17:55:47,822 : INFO : PROGRESS: fresh_shard size=10000
2018-04-29 17:55:52,892 : INFO : PROGRESS: fresh_shard size=20000
2018-04-29 17:55:56,233 : INFO : PROGRESS: fresh_shard size=30000
2018-04-29 17:55:56,985 : INFO : creating sparse index
2018-04-29 17:55:56,986 : INFO : creating sparse matrix from corpus
2018-04-29 17:55:56,987 : INFO : PROGRESS: at document #0/32768
2018-04-29 17:56:03,380 : INFO : PROGRESS: at document #10000/32768
2018-04-29 17:56:09,667 : INFO : PROGRESS: at document #20000/32768
2018-04-29 17:56:12,513 : INFO : PROGRESS: at document #30000/32768


CPU times: user 4min 58s, sys: 2.48 s, total: 5min
Wall time: 5min 8s


In [15]:
########################### load tfidf index
tfidf_index = gensim.similarities.Similarity.load('py_models/TFIDF_lem10_index')

2018-04-29 17:57:41,123 : INFO : loading Similarity object from py_models/TFIDF_lem10_index
2018-04-29 17:57:41,123 : INFO : loaded py_models/TFIDF_lem10_index


In [16]:
%%time

# calculate similarity for all 
tfidf_index.num_best= None
i#######################         SIMILIRATIES SCORES AND RANKING    ###########################
results = copy.deepcopy(query_truth)
### lists of list (indexing from 1, dummy element)
sim_list = [0]
rank_list = [0]
queries = [0]
for i,query in enumerate(files_to_tokens('LegalAdhocTask/q*.txt')):
    queries.append(query)
    # inside the square bracket determines query representation
    
    sims = tfidf_index[tfidf[dictionary.doc2bow(query)]]
    sim_list.append(sims)
    # rank of every document wrt similarity
    ranks = rankdata(sims, method='ordinal')
    ranks= len(ranks)+1 - ranks 
    rank_list.append(ranks)
    
    # update the query truth tuples with similarity score and the ranks
    for x in results[str(i+1)]:
        x.append(sims[filenames.index(x[0]+'.txt')])
        x.append(ranks[filenames.index(x[0]+'.txt')])
        #x.append(common_words(filenames.index(x[0]+'.txt'), tfidf_corpus, query))
    
    # sort wrt relevance(from truth) and then ranks(from our model)
    #tfidf_results[str(i+1)].sort(key=lambda x: (-int(x[1]),x[3]))     
    
        

2018-04-29 17:57:43,118 : INFO : loading SparseMatrixSimilarity object from py_models/tfidf_lem10.index.0
2018-04-29 17:57:43,119 : INFO : loading index from py_models/tfidf_lem10.index.0.index.npy with mmap=r
2018-04-29 17:57:43,120 : INFO : loaded py_models/tfidf_lem10.index.0
2018-04-29 17:57:43,157 : INFO : loading SparseMatrixSimilarity object from py_models/tfidf_lem10.index.1
2018-04-29 17:57:43,203 : INFO : loaded py_models/tfidf_lem10.index.1
2018-04-29 17:57:43,237 : INFO : loading SparseMatrixSimilarity object from py_models/tfidf_lem10.index.2
2018-04-29 17:57:43,238 : INFO : loading index from py_models/tfidf_lem10.index.2.index.npy with mmap=r
2018-04-29 17:57:43,240 : INFO : loaded py_models/tfidf_lem10.index.2
2018-04-29 17:57:43,282 : INFO : loading SparseMatrixSimilarity object from py_models/tfidf_lem10.index.3
2018-04-29 17:57:43,338 : INFO : loaded py_models/tfidf_lem10.index.3
2018-04-29 17:57:43,372 : INFO : loading SparseMatrixSimilarity object from py_models/tf

CPU times: user 6.72 s, sys: 240 ms, total: 6.96 s
Wall time: 7.13 s


In [17]:
results

{'1': [['ConsumerCourt_DCDRC_100385', '1', 0.16185194, 205],
  ['ConsumerCourt_DCDRC_106530', '0', 0.066299856, 2049],
  ['ConsumerCourt_DCDRC_107608', '1', 0.08884313, 1231],
  ['ConsumerCourt_DCDRC_114291', '1', 0.12928788, 532],
  ['ConsumerCourt_DCDRC_114382', '1', 0.10607624, 893],
  ['ConsumerCourt_DCDRC_118185', '1', 0.09642149, 1088],
  ['ConsumerCourt_DCDRC_130318', '0', 0.1199432, 653],
  ['ConsumerCourt_DCDRC_130570', '1', 0.27709824, 7],
  ['ConsumerCourt_DCDRC_131146', '1', 0.06343228, 2197],
  ['ConsumerCourt_DCDRC_131717', '1', 0.08180869, 1438],
  ['ConsumerCourt_DCDRC_131741', '1', 0.07304419, 1746],
  ['ConsumerCourt_DCDRC_131818', '1', 0.09741112, 1068],
  ['ConsumerCourt_DCDRC_131950', '0', 0.05442484, 2824],
  ['ConsumerCourt_DCDRC_131972', '0', 0.08816901, 1251],
  ['ConsumerCourt_DCDRC_132932', '0', 0.06848099, 1925],
  ['ConsumerCourt_DCDRC_133592', '1', 0.1699525, 155],
  ['ConsumerCourt_DCDRC_134386', '0', 0.33104795, 1],
  ['ConsumerCourt_DCDRC_135474', '1', 

In [18]:
model_name = 'TFIDF_lem10'

with open('py_results/' + model_name + 'sim_list.pkl', 'wb') as output:
    pickle.dump(sim_list, output, pickle.HIGHEST_PROTOCOL)
with open('py_results/' + model_name + 'rank_list.pkl', 'wb') as output:
    pickle.dump(rank_list, output, pickle.HIGHEST_PROTOCOL)


In [19]:
# Store results in 1st sheet


wb = Workbook()
dest_filename = 'result_excel/' + model_name + '.xlsx'
ws1 = wb.active
ws1.title = "ground_truth"
ws1.append(['Query','Filename', 'Relevance', 'Score', 'Rank'])
for key,value in results.iteritems():
    for i in value:
        ws1.append([int(key)]+ i)

wb.save(filename = dest_filename)

#Query wise (F score)

import pandas as pd

f = pd.ExcelFile(dest_filename)

from scipy.stats import hmean



places = 4
wb = load_workbook(dest_filename)
ws1 = wb.create_sheet(title="evaluation_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])

k = [1.0, 5.0 ,10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]
for sheet in f.sheet_names:
    if(sheet == "evaluation"):
        continue
    for q in np.arange(1,11):

        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total_1 = (float)(x[x['Relevance'] == 1].shape[0])
        total_0 = (float)(x[x['Relevance'] == 0].shape[0])
        total = (float)(x.shape[0])
        
        ####  repitition due to complication in considering both 1 & 0 relevance
        # precision 1
        row = [q, 1, 'P', model_name ]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/i)
        ws1.append(row);
        p1 = row;
        # precision 0
        row = [q, 0, 'P', model_name ]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/i)    
        ws1.append(row);
        p0 = row;
        #precision 10
        row = [q, 10, 'P', model_name]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/i)  
        ws1.append(row);
        p10 = row;  
        
        
        # recall 1
        row = [q, 1, 'R', model_name]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/total_1)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r1 = row;    
        # recall 0
        row = [q, 0, 'R', model_name ]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/total_0)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r0 = row;
        #recall 10
        row = [q, 10, 'R', model_name ]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/total)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r10 = row;      


        # F 1
        row = [q, 1, 'F', model_name ]
        for i in range(-len(k), 0, 1):
            if p1[i] == 0.0 :
                row.append(0.0)
            else:
                row.append(hmean([p1[i],r1[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 0
        row = [q, 0, 'F', model_name]
        for i in range(-len(k), 0, 1):
            if p0[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p0[i],r0[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 10
        row = [q, 10, 'F', model_name]
        for i in range(-len(k), 0, 1):
            if p10[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p10[i],r10[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);
        
wb.save(filename = dest_filename)


#Query wise Average precision
# precision sum
def p_sum(z):
    z = z.copy()  
    z.sort_values(inplace=True)
    result = 0
    for i,val in enumerate(z):
        result += (i+1)/float(val) 
    return result


import pandas as pd

f = pd.ExcelFile(dest_filename)

from scipy.stats import hmean


places = 4
wb = load_workbook(dest_filename)
ws1 = wb.create_sheet(title="AP_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])


for sheet in f.sheet_names:    
    if(sheet == "evaluation"or sheet == "evaluation_Qwise"):
        continue
    for q in np.arange(1,11):
        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total = {}
        total[1] = (float)(x[x['Relevance'] == 1].shape[0])
        total[0] = (float)(x[x['Relevance'] == 0].shape[0])
        total[10] = (float)(x.shape[0])
        
        for rel in [1,0]:
            # precision 1
            row = [q, rel, 'AP', model_name ]
            for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
                row.append(p_sum(x[np.logical_and(x['Relevance'] == rel, x['Rank'] <= i)]['Rank'])/total[rel])
            ws1.append(row);

        row = [q, 10, 'AP', model_name ]
        for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
            row.append(p_sum(x[x['Rank'] <= i]['Rank'])/total[10])
        ws1.append(row);
wb.save(filename = dest_filename)

In [20]:
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
      truncate_sheet : truncate (remove and recreate) [sheet_name]
                       before writing DataFrame to Excel file
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel()`
                        [can be dictionary]

    Returns: None
    """
    from openpyxl import load_workbook

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl')

    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)

        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row

        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)

        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, index= False, header = False, **to_excel_kwargs)

    # save the workbook
    writer.save()

    
f = pd.ExcelFile(dest_filename)
sec_dest = 'result_excel/all_results.xlsx'
f2 = pd.ExcelFile(sec_dest)
append_df_to_excel(sec_dest, f.parse(1), sheet_name= f2.sheet_names[0])
append_df_to_excel(sec_dest, f.parse(2), sheet_name= f2.sheet_names[1])

In [ ]:
############################################################

In [ ]:
def Top_x(x, R):
    R = list(R)
    x_names = []
    for i in range(x):
        x_names.append(filenames[R.index(i + 1)].strip('.txt'))
    return x_names
    
    

In [ ]:
def score_stat(word, freq, index):
    index_doc = dict(bm25.corpus[index])
    word = dictionary.doc2bow([word])[0][0]
    idf = bm25.idf[word] if bm25.idf[word] >= 0 else EPSILON * average_idf
    score = freq*((idf * index_doc[word] * (PARAM_K1 + 1)
              / (index_doc[word] + PARAM_K1 * (1 - PARAM_B + PARAM_B * doc_len[index] / bm25.avgdl))))
    return (index_doc[word], score)

In [ ]:
wb = Workbook()
dest_filename = 'Temp.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)
def common_words(q, F):
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    index = filenames.index(F + '.txt')
    index_doc = dict(bm25.corpus[index])
    row = []
    for word, freq in  zip(q_words, q_freq):
        w = dictionary.doc2bow([word])[0][0]
        if w in index_doc: 
            f, s = score_stat(word, freq, index)
            row.append([word + ', ' + str(round(bm25.idf[w], 2)), freq, f,  round( (s*100)/sim_list[q][index], 1)])  
    row.sort(key = lambda x: (x[3],x[1],x[2]), reverse = True)
    wb = load_workbook('Temp.xlsx')
    ws = wb.create_sheet(title=str(q) + F)
    ws.append(['word,idf', 'Query freq', 'freq', '% score'])
    for i in row:
        ws.append(i)
    wb.save(filename = 'Temp.xlsx')


In [ ]:
f = open("toberead.txt", 'r') 
count = 0
q = 1
for i in f:
    if i == '\n':
        continue
    count = count + 1
    common_words(q, i.strip())
    if count % 5 == 0:
        q = q + 1


In [ ]:
for i in files_to_tokens(full_filenames[filenames.index(doc + '.txt')]):
    for j in i:
        print j

In [ ]:
doc = 'ConsumerCourt_SCDRC_30176'
corpus[]

In [ ]:
common_words(1, 'ConsumerCourt_DCDRC_41588')

In [ ]:
df = pd.DataFrame(index= None, columns=['words','pair'])


In [ ]:
df

In [ ]:
print df

In [ ]:
df.append(['lol', ('we','wew')])

In [ ]:
def uncommon_words(q, filename)

In [ ]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'Explore_BM25.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)

wb = load_workbook('Explore_BM25.xlsx')

for q in range(1, 11):
    ws = wb.create_sheet(title="q" + str(q))
    #x_names = Top_x(10, rank_list[q])
    top_row = ['Doc Name', 'Rel', 'Doc Length', ' Rank', 'Score']
    sec_row = [' ']*5
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    ws.append(top_row + [i[0]  for i in temp])
    ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    
    #continue
    doc_list = [i[0] for i in query_truth[str(q)]] + Top_x(10, rank_list[q])
    doc_list = list((set(doc_list)))
    doc_list.sort(key = lambda doc : rank_list[q][filenames.index(doc + '.txt')])
    doc_row = []
    for doc in doc_list:
        temp  = [doc]
        rel_dic = dict(query_truth[str(q)])
        if doc in rel_dic:
            temp.append(rel_dic[doc])
        else:
            temp.append('Not')
        temp.append(doc_len[filenames.index(doc + '.txt')])
        temp.append(rank_list[q][filenames.index(doc + '.txt')])
        temp.append(round(sim_list[q][filenames.index(doc + '.txt')], 2))
        doc_row.append(temp)

    for row in doc_row:
        index = filenames.index(row[0] + '.txt')
        index_doc = dict(bm25.corpus[index])
        
        for word, freq in  zip(q_words, q_freq):
            if dictionary.doc2bow([word])[0][0] in index_doc: 
                f, s = score_stat(word, freq, index)
                row.append( str(f) + ', ' +  str(round( (s*100)/sim_list[q][index], 1) ))
            else:
                row.append('X')
        ws.append(row)
        ws.append([])        
wb.save(filename = dest_filename)

In [ ]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'Explore_BM25_uncom.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)



wb = load_workbook('Explore_BM25_uncom.xlsx')

for q in range(1, 11):
    ws = wb.create_sheet(title="q" + str(q))
    #x_names = Top_x(10, rank_list[q])
    top_row = ['Doc Name', 'Rel', 'Doc Length', ' Rank', 'Score']
    sec_row = [' ']*5
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    ws.append(top_row + [i[0]  for i in temp])
    ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    
    #continue
    doc_list = [i[0] for i in query_truth[str(q)]] + Top_x(10, rank_list[q])
    doc_list = list((set(doc_list)))
    doc_list.sort(key = lambda doc : rank_list[q][filenames.index(doc + '.txt')])
    doc_row = []
    for doc in doc_list:
        temp  = [doc]
        rel_dic = dict(query_truth[str(q)])
        if doc in rel_dic:
            temp.append(rel_dic[doc])
        else:
            temp.append('Not')
        temp.append(doc_len[filenames.index(doc + '.txt')])
        temp.append(rank_list[q][filenames.index(doc + '.txt')])
        temp.append(round(sim_list[q][filenames.index(doc + '.txt')], 2))
        doc_row.append(temp)

    for row in doc_row:
        index = filenames.index(row[0] + '.txt')
        index_doc = dict(bm25.corpus[index])
        
#         for word, freq in  zip(q_words, q_freq):
#             if dictionary.doc2bow([word])[0][0] in index_doc: 
#                 f, s = score_stat(word, freq, index)
#                 row.append( str(f) + ', ' +  str(round( (s*100)/sim_list[q][index], 1) ))
#             else:
#                 row.append('X')
        #ws.append(row)
        
        sec_row = [' ']*5
        temp = []
        for word, freq in index_doc.iteritems():
            if dictionary[word] not in q_words:
                temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
        ws.append(row + [i[0]  for i in temp])
        ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
        ws.append([])        
wb.save(filename = dest_filename)

In [ ]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'Explore_BM25_all.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)



wb = load_workbook('Explore_BM25_all.xlsx')

for q in range(1, 11):
    ws = wb.create_sheet(title="q" + str(q))
    #x_names = Top_x(10, rank_list[q])
    top_row = ['Doc Name', 'Rel', 'Doc Length', ' Rank', 'Score']
    sec_row = [' ']*5
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    ws.append(top_row + [i[0]  for i in temp])
    ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    
    #continue
    doc_list = [i[0] for i in query_truth[str(q)]] + Top_x(10, rank_list[q])
    doc_list = list((set(doc_list)))
    doc_list.sort(key = lambda doc : rank_list[q][filenames.index(doc + '.txt')])
    doc_row = []
    for doc in doc_list:
        temp  = [doc]
        rel_dic = dict(query_truth[str(q)])
        if doc in rel_dic:
            temp.append(rel_dic[doc])
        else:
            temp.append('Not')
        temp.append(doc_len[filenames.index(doc + '.txt')])
        temp.append(rank_list[q][filenames.index(doc + '.txt')])
        temp.append(round(sim_list[q][filenames.index(doc + '.txt')], 2))
        doc_row.append(temp)

    for row in doc_row:
        index = filenames.index(row[0] + '.txt')
        index_doc = dict(bm25.corpus[index])
        
        for word, freq in  zip(q_words, q_freq):
            if dictionary.doc2bow([word])[0][0] in index_doc: 
                f, s = score_stat(word, freq, index)
                row.append( str(f) + ', ' +  str(round( (s*100)/sim_list[q][index], 1) ))
            else:
                row.append('X')
        ws.append(row)
        
        sec_row = [' ']*5
        temp = []
        for word, freq in index_doc.iteritems():
            if dictionary[word] not in q_words:
                temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
        ws.append(sec_row + [i[0]  for i in temp])
        ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
        ws.append([])        
wb.save(filename = dest_filename)